# 1. Import the necessary libraries for you to be able to query 4square API

In [4]:
import requests
import json
import os
import pandas as pd


# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [5]:
# Define your API key
api_key = "fsq3ua10RsqeNLTtC0b7F/FZ5nj6iBJ+ZD1eFUcTWsmxBqU="

# Define request details
url = "https://api.foursquare.com/v3/places/search"
headers = {
    "Authorization": api_key
}
params_10 = {
    "query": "coffee",
    "near": "Madrid, Spain",
    "limit": 10
}

# Make the request
response_10 = requests.get(url, headers=headers, params=params_10)
coffee_10 = response_10.json()

# Display results
if "results" in coffee_10:
    for place in coffee_10["results"]:
        name = place.get("name", "No name")
        address = place.get("location", {}).get("formatted_address", "No address")
        print(f"{name} - {address}")
else:
    print("❌ Error or no results:")
    print(json.dumps(coffee_10, indent=2))


Bucólico Café - Calle Barbieri, 4, 28004 Madrid Comunidad de Madrid
Chocolatería San Ginés - Pasadizo de San Ginés, 5 (C. Arenal), 28013 Madrid Comunidad de Madrid
Valdemeso - Mesonero Romanos, 8, 28013 Madrid Comunidad de Madrid
Oink Jamón Ibérico - Calle Gran Vía, 22, 28013 Madrid Comunidad de Madrid
Brunch Club - Calle del Barco, 15, 28004 Madrid Comunidad de Madrid
Ruda Café - Calle Ruda, 11, 28005 Madrid Comunidad de Madrid
Osom Cantine - Calle de Santa Teresa, 2, local 2, 28014 Madrid Comunidad de Madrid
HanSo Café - Calle del Pez, 20, 28004 Madrid Comunidad de Madrid
Caravan Cafe - Calle de Lavapiés, 32, Madrid Comunidad de Madrid
Federal Café 2 - Plaza Conde de Barajas, 3, 28005 Madrid Comunidad de Madrid


# 3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [6]:
params_5 = {
    "query": "coffee",
    "ll": "40.3925,-3.6984",
    "limit": 5
}

response_5 = requests.get(url, headers=headers, params=params_5)
coffee_5 = response_5.json()

# Print names and coordinates
for place in coffee_5.get("results", []):
    name = place.get("name", "No name")
    lat = place.get("geocodes", {}).get("main", {}).get("latitude")
    lng = place.get("geocodes", {}).get("main", {}).get("longitude")
    print(f"{name} -> ({lat}, {lng})")


Cantina Matadero -> (40.391515, -3.696042)
Dot Café Bar -> (40.396051, -3.6988)
Salon de Te al Yabal -> (40.39908, -3.700202)
Cafetería Churrería Legazpi -> (40.391174, -3.69435)
BBIG Delice Café -> (40.396268, -3.693275)


# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [7]:
def foursquare_places(venue, coordinates):
    url = "https://api.foursquare.com/v3/places/search"
    headers = {
        "Authorization": api_key
    }
    params = {
        "query": venue,
        "ll": coordinates,
        "limit": 10
    }
    response = requests.get(url, headers=headers, params=params)
    results = response.json()
    
    output = []
    for place in results.get("results", []):
        output.append({
            "name": place.get("name"),
            "lat": place.get("geocodes", {}).get("main", {}).get("latitude"),
            "long": place.get("geocodes", {}).get("main", {}).get("longitude")
        })
    return output

# Example usage:
results = foursquare_places("starbucks", "40.3925,-3.6984")
pd.DataFrame(results)


,name,lat,long
0,Starbucks,40.391357,-3.701159


# 5. Export it into data folder as a json: `data/coffe_shops.json`

In [8]:
# Make sure the "data" folder exists
os.makedirs("data", exist_ok=True)

# Save results to JSON
with open("data/coffee_shops.json", "w") as f:
    json.dump(results, f, indent=4)

print("✅ Exported to data/coffee_shops.json")


✅ Exported to data/coffee_shops.json


# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [13]:
from pymongo import MongoClient
import json

# Connect to local MongoDB
client = MongoClient("mongodb://localhost:27017/")

# Access database and collection
db = client["ironhack"]
collection = db["ironcoffee"]

# Load data from JSON file
with open("data/coffee_shops.json", "r") as f:
    coffee_data = json.load(f)

# Insert into MongoDB
if isinstance(coffee_data, list):
    collection.insert_many(coffee_data)
else:
    collection.insert_one(coffee_data)

print("✅ Data inserted into MongoDB collection 'ironcoffee'")

# Count how many documents are in the collection
print("Total documents in 'ironcoffee':", collection.count_documents({}))

# Preview the first document
print(collection.find_one())



ModuleNotFoundError: No module named 'pymongo'